Задание 1: отобрать только числовые признаки и обучить модель XGBoost с параметром booster = gbtree. Обучать алгоритм до тех пор, пока метрика качества не перестанет улучшаться на валидационной выборке в течение определенного числа итераций (выбрать значение самостоятельно).

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
df_train = pd.read_csv(
    "data/assignment_2_train.csv"
)

df_test = pd.read_csv(
    "data/assignment_2_test.csv"
)

print("gf_train.shape = {} rows, {} cols".format(*df_train.shape))
print("df_test.shape = {} rows, {} cols".format(*df_test.shape))
df_train.head(n=2)

FileNotFoundError: [Errno 2] File data/assignment_2_train.csv does not exist: 'data/assignment_2_train.csv'

In [ ]:
df_train['isFraud'].unique()

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(df_train, test_size=0.33,  random_state=42)

In [ ]:
y_train = df_train['isFraud']
y_valid = df_valid['isFraud']
y_test = df_test['isFraud']

df_train = df_train.drop(["isFraud", "TransactionID"], axis=1)
df_valid = df_valid.drop(["isFraud", "TransactionID"], axis=1)
df_test = df_test.drop(["isFraud", "TransactionID"], axis=1)

print("df_train.shape = {} rows, {} cols".format(*df_train.shape))
print("df_train.shape = {} rows, {} cols".format(*df_valid.shape))
print("df_test.shape = {} rows, {} cols".format(*df_test.shape))

In [ ]:
num_df_train = df_train.select_dtypes(include=[np.number])
num_df_valid = df_valid.select_dtypes(include=[np.number])
num_df_test = df_test.select_dtypes(include=[np.number])
print(f"count of num_df_train {num_df_train.shape[1]}")
print(f"count of num_df_valid {num_df_valid.shape[1]}")
print(f"count of num_df_test {num_df_test.shape[1]}")

num_df_train.columns

In [ ]:
%%time

dtrain = xgb.DMatrix(data=num_df_train, label=y_train)
print('*')
dvalid = xgb.DMatrix(data=num_df_valid, label=y_valid)
print('*')
dtest_xgb_num = xgb.DMatrix(data=num_df_test)

In [ ]:
params = {
    "booster": "gbtree",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 1000,
    "reg_lambda": 100,
    "max_depth": 4,
    "gamma": 10,
    "nthread": 6,
    "seed": 27
}

In [ ]:
model_xgb_num = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=1000,
    early_stopping_rounds=10,
    evals=[(dtrain, "train"), (dvalid, "valid")],
    verbose_eval=10,
    maximize=True,
)

Задание 2: обработать категориальные признаки любым способом (который вы знаете) и добавить их к данным. Выполнить задание 1.

In [ ]:
сategorical_feature = list(set(df_train.columns) - set(num_df_train.columns))
сategorical_feature

In [ ]:
categorical_df_train = df_train.copy()
categorical_df_valid = df_valid.copy()
categorical_df_test = df_test.copy()

for i in range(len(сategorical_feature)):
    print(f'feature {сategorical_feature[i]}')
    сategorical_train = df_train[сategorical_feature[i]].unique()
    сategorical_valid = df_valid[сategorical_feature[i]].unique()
    сategorical_test = df_test[сategorical_feature[i]].unique()
    #print(f'сategorical train {сategorical_train}')
    #print(f'сategorical test {сategorical_test}')
    
    cat_in_feature = list(set(сategorical_train) & set(сategorical_valid) & set(сategorical_test))
    for cat in cat_in_feature:
        dummie_feature_name = f'{сategorical_feature[i]}_{cat}'
        #print(dummie_feature_name)
        
        if str(cat) != 'nan':
            print(f'    dummie feature name: {dummie_feature_name}')
            categorical_df_train[f'{сategorical_feature[i]}_{cat}'] = 0
            categorical_df_train.loc[categorical_df_train.loc[:,сategorical_feature[i]]==cat, 
                                                          [dummie_feature_name]] = 1      
            
            categorical_df_valid[f'{сategorical_feature[i]}_{cat}'] = 0
            categorical_df_valid.loc[categorical_df_valid.loc[:,сategorical_feature[i]]==cat, 
                                                          [dummie_feature_name]] = 1  
            
            categorical_df_test[f'{сategorical_feature[i]}_{cat}'] = 0
            categorical_df_test.loc[categorical_df_test.loc[:,сategorical_feature[i]]==cat, 
                                                          [dummie_feature_name]] = 1 
            
    categorical_df_train.drop([сategorical_feature[i]], axis='columns', inplace=True)
    categorical_df_valid.drop([сategorical_feature[i]], axis='columns', inplace=True)
    categorical_df_test.drop([сategorical_feature[i]], axis='columns', inplace=True)
    
print(f"count of categorical_df_train {categorical_df_train.shape}")
print(f"count of categorical_df_valid {categorical_df_valid.shape}")
print(f"count of categorical_df_test {categorical_df_test.shape}")

In [ ]:
%%time

dtrain = xgb.DMatrix(data=categorical_df_train, label=y_train)
print('*')
dvalid = xgb.DMatrix(data=categorical_df_valid, label=y_valid)
print('*')
dtest_xgb_num_cat = xgb.DMatrix(data=categorical_df_test)

In [ ]:
model_xgb_num_cat = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=10000,
    early_stopping_rounds=30,
    evals=[(dtrain, "train"), (dvalid, "valid")],
    verbose_eval=50,
    maximize=True,
)

Задание 4: для числовых признаков обучить модель LightGBM. Обучать алгоритм до тех пор, пока метрика качества не перестанет улучшаться на валидационной выборке в течение определенного числа итераций (выбрать значение самостоятельно).

In [ ]:
import lightgbm as lgb

In [ ]:
params = {
    "boosting_type": "goss",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 200,
    "n_jobs": 6,
    "seed": 27
}

In [ ]:
dtrain = lgb.Dataset(
    data=num_df_train, label=y_train
)
dvalid = lgb.Dataset(
    data=num_df_valid, label=y_valid
)
dtest_lgb_num = lgb.Dataset(
    data=num_df_test
)

model_lgb_num = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=10000,
    valid_sets=[dtrain, dvalid],
    categorical_feature="auto",
    early_stopping_rounds=50,
    verbose_eval=10
)

Задание 5: обработать категориальные признаки любым способом (который вы знаете) и добавить их к данным. Выполнить задание 4.

In [ ]:
dtrain = lgb.Dataset(
    data=categorical_df_train, label=y_train
)
dvalid = lgb.Dataset(
    data=categorical_df_valid, label=y_valid
)
dtest_lgb_num_cat = lgb.Dataset(
    data=categorical_df_test
)

model_lgb_num_cat = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=10000,
    valid_sets=[dtrain, dvalid],
    categorical_feature="auto",
    early_stopping_rounds=50,
    verbose_eval=20
)

Задание 6: обработать категориальные признаки встроенным методом в LightGBM. Выполнить задание 4. Сделать выводы о качестве работы алгоритма, по сравнению с пунктом 5.

In [ ]:
сategorical_feature

In [ ]:
df_train.columns

In [ ]:
catlgd_df_train = df_train.copy()
catlgd_df_valid = df_valid.copy()
catlgd_df_test = df_test.copy()

for c in сategorical_feature:
    catlgd_df_train[c] = catlgd_df_train[c].astype('category')
    catlgd_df_valid[c] = catlgd_df_valid[c].astype('category')
    catlgd_df_test[c] = catlgd_df_test[c].astype('category')


dtrain = lgb.Dataset(
    data=catlgd_df_train, label=y_train, categorical_feature=сategorical_feature
)
dvalid = lgb.Dataset(
    data=catlgd_df_valid, label=y_valid, categorical_feature=сategorical_feature
)
dtest_lgb_num_catlgb = lgb.Dataset(
    data=catlgd_df_test, categorical_feature=сategorical_feature
)

In [ ]:
model_lgb_num_catlgb = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=10000,
    valid_sets=[dtrain, dvalid],
    early_stopping_rounds=50,
    verbose_eval=10
)

Задание 7: для числовых признаков обучить модель CatBoost. Обучать алгоритм до тех пор, пока метрика качества не перестанет улучшаться на валидационной выборке в течение определенного числа итераций (выбрать значение самостоятельно).

In [ ]:
import catboost as cb

In [ ]:
cb_params = {
    "n_estimators": 800,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 20,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

In [ ]:
model_cb_num = cb.CatBoostClassifier(**cb_params)
model_cb_num.fit(num_df_train, 
                 y_train, 
                 eval_set=[(num_df_train, y_train), (num_df_valid, y_valid)],
                 plot=True
                )

Задание 8: обработать категориальные признаки любым способом (который вы знаете) и добавить их к данным. Выполнить задание 7.

In [ ]:
model_cb_num_cat = cb.CatBoostClassifier(**cb_params)
model_cb_num_cat.fit(categorical_df_train, 
                 y_train, 
                 eval_set=[(categorical_df_train, y_train), (categorical_df_valid, y_valid)],
                 plot=True
                )

Задание 9: обработать категориальные признаки встроенным методом в CatBoost. Выполнить задание 7. Сделать выводы о качестве работы алгоритма, по сравнению с пунктом 8.

In [ ]:
catcb_df_train = df_train.copy()
catcb_df_valid = df_valid.copy()
catcb_df_test = df_test.copy()

for c in сategorical_feature:
    catcb_df_train[c] = catcb_df_train[c].astype('str')
    catcb_df_valid[c] = catcb_df_valid[c].astype('str')
    catcb_df_test[c] = catcb_df_test[c].astype('str')


train_pool = cb.Pool(catcb_df_train, y_train, cat_features=сategorical_feature)
valid_pool = cb.Pool(catcb_df_valid, y_valid, cat_features=сategorical_feature)
test_pool = cb.Pool(catcb_df_test,cat_features=сategorical_feature)

In [ ]:
model_cb_num_catcb = cb.CatBoostClassifier(**cb_params)
model_cb_num_catcb.fit(
                 train_pool,
                 eval_set=[valid_pool],
                 plot=True
                )

In [ ]:
from sklearn.metrics import roc_curve, auc

fig = plt.figure(figsize=(25, 15))

#------------------------------------------------------------------------------
#xgb
pred = model_xgb_num.predict(dtest_xgb_num)
fpr, tpr, _ = roc_curve(y_test, pred)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='dtest_xgb_num (area = %0.6f)' % roc_auc)

pred = model_xgb_num_cat.predict(dtest_xgb_num_cat)
fpr, tpr, _ = roc_curve(y_test, pred)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='dtest_xgb_num_cat (area = %0.6f)' % roc_auc)
#------------------------------------------------------------------------------
#lgb
pred = model_lgb_num.predict(num_df_test)
fpr, tpr, _ = roc_curve(y_test, pred)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='dtest_lgb_num (area = %0.6f)' % roc_auc)


pred = model_lgb_num_cat.predict(categorical_df_test)
fpr, tpr, _ = roc_curve(y_test, pred)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='dtest_lgb_num_cat (area = %0.6f)' % roc_auc)

pred = model_lgb_num_catlgb.predict(catlgd_df_test)
fpr, tpr, _ = roc_curve(y_test, pred)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='dtest_lgb_num_catlgb (area = %0.6f)' % roc_auc)
#------------------------------------------------------------------------------
#cb
pred = model_cb_num.predict(num_df_test)
fpr, tpr, _ = roc_curve(y_test, pred)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='dtest_cb_num (area = %0.6f)' % roc_auc)

pred = model_cb_num_cat.predict(categorical_df_test)
fpr, tpr, _ = roc_curve(y_test, pred)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='dtest_cb_num_cat (area = %0.6f)' % roc_auc)

pred = model_cb_num_catcb.predict(test_pool)
fpr, tpr, _ = roc_curve(y_test, pred)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='dtest_cb_num_catcb (area = %0.6f)' % roc_auc)


plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

Наилучший оказался lightgbm. Он держит марку даже с недостающими категориальными данными. Работать с категориальными признаками выгоднее как с дами переменой т.к. средства lightgbm при работе с категориальными признаками понизили точность. Catboost просто не сработал, хотя опять же с категориями лучше работать как с дами. lightgbm и XGBoost практически одинаковую точность выдали, но lightgbm и быстрее отрабатывает.